# parallel coordinator

In [1]:
%%js
$("div.input").hide();
element.append(`<button type="button" 
                   onclick='$("div.input").toggle(1000)'
                   class="btn btn-primary active">toggle code</button>`)


<IPython.core.display.Javascript object>

In [192]:
import numpy as np
import struct
import pandas as pd
def extract_labels(mnist_label_file_path):
    with open(mnist_label_file_path, "rb") as f:
        magic_number = struct.unpack('i',f.read(4))
        num_items = struct.unpack('i',f.read(4))
        labels = reduce(lambda a,b:a+[ord(b)],f.read(),[])
    return labels
def extract_images(images_file_path):
    with open(images_file_path, "rb") as f:
        magic_number = f.read(4)
        num_images = f.read(4)
        num_rows = f.read(4)
        num_cols = f.read(4)
        rimg = f.read(784)
        rimgs = []
        while rimg:
            rimgs = rimgs + [rimg]
            rimg = f.read(784)
        imgs = reduce(lambda imgs,rimg:imgs+[struct.unpack('=784B',rimg)],rimgs,[])
    return imgs
imgs = extract_images('/Users/huynh/github/uci_dataset/mnist/train-images-idx3-ubyte')
labels = extract_labels('/Users/huynh/github/uci_dataset/mnist/train-labels-idx1-ubyte')
data = [[l] + list(imgs[i]) for i,l in enumerate(labels)]
data_df = pd.DataFrame(data,columns=['label']+['p'+str(i) for i in range(784)])
#try to avoid trouble with github
data_df[:30000].to_csv('/Users/huynh/github/uci_dataset/mnist/train_1.csv',index=None)
data_df[30000:].to_csv('/Users/huynh/github/uci_dataset/mnist/train_2.csv',index=None)

In [191]:
test_imgs = extract_images('/Users/huynh/github/uci_dataset/mnist/t10k-images-idx3-ubyte')
test_labels = extract_labels('/Users/huynh/github/uci_dataset/mnist/t10k-labels-idx1-ubyte')
test_data = [[l] + list(test_imgs[i]) for i,l in enumerate(test_labels)]
test_data_df = pd.DataFrame(test_data,columns=['label']+['p'+str(i) for i in range(784)])

In [ ]:
print(test_data_df.shape, data_df.shape)

In [7]:
%%js
require.config({
  paths: {
    d3: "http://d3js.org/d3.v4.min"
  }
});

<IPython.core.display.Javascript object>

In [25]:
%%js
require(["d3"], function(d3) {
    element.append("<svg></svg>");
    let svg = d3.select('svg');
    element.append(`<p id="log"></p><p id="log"></p>`)
    const log = (msg)=>{$('#log').append(msg)}
    const track = (msg)=>{$('#log').append(msg)}
    log('bs')

});

<IPython.core.display.Javascript object>

In [ ]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/BgjmhbkPVAI" 
    frameborder="0" allowfullscreen></iframe>

Metropolis-Hastings [@metropolis1953equation; @hastings1970monte].

  #### Notes

  In conditional inference, we infer $z$ in $p(z, \\beta\mid x)$ while fixing inference over $\\beta$ using another
  distribution $q(\\beta)$.
  To calculate the acceptance ratio, `MetropolisHastings` uses an
  estimate of the marginal density,

  $p(x, z) = \mathbb{E}_{q(\\beta)} [ p(x, z, \\beta) ]
            \\approx p(x, z, \\beta^*)$

  leveraging a single Monte Carlo sample, where $\\beta^* \sim
  q(\\beta)$. This is unbiased (and therefore asymptotically exact as a
  pseudo-marginal method) if $q(\\beta) = p(\\beta \mid x)$.

  #### Examples

  ```python
  mu = Normal(loc=0.0, scale=1.0)
  x = Normal(loc=mu, scale=1.0, sample_shape=10)

  qmu = Empirical(tf.Variable(tf.zeros(500)))
  proposal_mu = Normal(loc=mu, scale=0.5)
  inference = ed.MetropolisHastings({mu: qmu}, {mu: proposal_mu},
                                    data={x: np.zeros(10, dtype=np.float32)})
  ```